In [19]:
import pandas as pd
import numpy as np
import cv2
import os
import json

In [2]:
df = pd.read_csv("../data/sealion/Train/train.csv")

In [3]:
df.head()

,train_id,adult_males,subadult_males,adult_females,juveniles,pups
0,0,62,12,486,42,344
1,1,2,20,0,12,0
2,2,2,0,38,20,0
3,3,8,5,41,7,38
4,4,6,9,2,0,0


In [4]:
sealion_types = ["adult_males", 
    "subadult_males",
    "adult_females",
    "juveniles",
    "pups"]

In [5]:
df_sum = df.sum()

In [6]:
counts = dict()
total = 0
for s in sealion_types:
    counts[s] = df_sum[s]
    total += 1/counts[s]

In [7]:
probas = dict()
for s, c in counts.items():
    probas[s] = (1/counts[s])/total

In [8]:
background_proba = np.min(list(probas.values()))/10

In [9]:
probas

{'adult_females': 0.048142699511667254,
 'adult_males': 0.3351506883474506,
 'juveniles': 0.089826648353188876,
 'pups': 0.11096914409391793,
 'subadult_males': 0.41591081969377525}

In [10]:
from keras.preprocessing.image import Iterator

Using TensorFlow backend.


In [11]:
import keras.backend as K

In [121]:
class PatchIterator(Iterator):
    """Iterator yielding training samples
    :param root_dir: Directory containing training images, density map and sampling map.
    :param image_ids: Set of image ids to use to sample patches.
    :param x_mean: Mean per channel for input normalization
    :param x_std: Standard deviation per channel for input normalization
    :param y_scale: Scale to apply on the output density map
    :param output_counts: Indicate if the iterator should return only counts, otherwise density maps.
    :param n_samples_per_block: Number of patches to sample on each image.
    :param n_successive_samples: Number of samples to take on the same block when the block is loaded.
    :param target_size: Size of the patches sampled.
    :param batch_size: Number of patches sampled per batch
    :param shuffle: Boolean, whether to shuffle the data between epochs.
    :param seed: Random seed for data shuffling.
    :return batch_x, batch_x. 
        batch_x is a (batch_size, target_size[0], target_size[1], 3) array
        batch_x is a (batch_size, target_size[0], target_size[1], 5) array if output_counts is False
        otherwise, it is a (batch_size, 5) array.
    """

    def __init__(self, root_dir, image_ids,
                 x_mean, x_std, y_scale,
                 output_counts=False,
                 n_samples_per_block=12,
                 n_successive_samples=4,
                 target_size=(224, 224),
                 batch_size=8, shuffle=True, seed=42):
                 
        assert(n_samples_per_block % n_successive_samples == 0)
        assert(batch_size % n_successive_samples == 0)
        
        # Read the file containing the block status, ie if a block should be used or not
        with open(os.path.join(root_dir, "blocks_status.json"), "r") as jfile:
            blocks_status = json.load(jfile)
        
        # Fill with only valid blocks.
        self.image_ids = []
        self.block_ids = []
        for img_id in image_ids:
            blocks = blocks_status[str(img_id)]
            for bid, bstatus in enumerate(blocks):
                if bstatus:
                    self.image_ids.append(img_id)
                    self.block_ids.append(bid)
        
        self.x_mean = x_mean
        self.x_std = x_std
        self.y_scale = y_scale
        self.output_counts = output_counts
        self.root_dir = root_dir
        self.samples_per_block = n_samples_per_block
        self.n_successive_samples = n_successive_samples
        self.target_size = target_size
        self.n_blocks_per_batch = (n_samples_per_block // n_successive_samples)
        self.n_indices = len(self.image_ids) * self.n_blocks_per_batch
                 
        super(PatchIterator, self).__init__(self.n_indices, batch_size//self.n_successive_samples, shuffle, seed)
        
    def __normalize_input__(self, batch_x):
        return (batch_x - self.x_mean) / self.x_std
    
    def __normalize_output__(self, batch_y):
        return batch_y * self.y_scale
    
    def __normalize_sampling__(self, smap):
        w_edge = self.target_size[1]//2
        h_edge = self.target_size[0]//2
        # Set border to 0, it will avoid to deal with patches outside of the image
        smap[:, :w_edge] = 0
        smap[:, -w_edge:] = 0
        smap[:h_edge,:]=0
        smap[-h_edge:,:]=0
        # Normalize, to use it as a probability distribution
        return smap / np.sum(smap)

    def __sample__(self, img, dmap , smap, n):
        threshold_masked = 0.3 # if more than 30% of the patch is masked, reject it
        h, w, _ = img.shape
        pdistribution = self.__normalize_sampling__(smap).ravel()
        
        # There's a small risk that we can't find patches with enough non-masked pixels
        # so, we try several times (but we don't want to loop indefinitely)
        n_trys = 0
        i = 0
        while n_trys < 3:
            # Sample 10 * n random locations (we sample more because we may reject some)
            loc_indices = np.random.choice(h * w, size=(10 * n, ), replace=False, p=pdistribution)
            xs = np.zeros((n, ) + self.target_size + (3,))
            ys = np.zeros((n, ) + self.target_size + (5,))
            
            for loc_index in loc_indices:
                x = loc_index % w
                y = loc_index // w
                print("x, y: ", x, y)
                y_start = y - self.target_size[0] // 2
                y_end = y + self.target_size[0] // 2
                x_start = x - self.target_size[1] // 2
                x_end = x + self.target_size[1] // 2
                img_patch = img[y_start:y_end, x_start:x_end, :]
                masked_pixels = np.count_nonzero(img_patch == 0)
                total_pixels = img_patch.shape[0] * img_patch.shape[1]
                if masked_pixels/total_pixels < threshold_masked:
                    xs[i, ...] = img_patch
                    ys[i, ...] = dmap[y_start:y_end, x_start:x_end, :]
                    i += 1
                    if i == n:
                        return xs, ys
            n_trys += 1
        raise Exception("ERROR: Can't find non masked patches")
        
    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        # Keeps under lock only the mechanism which advances
        # the indexing of each batch.
        with self.lock:
            index_array, current_index, current_batch_size = next(self.index_generator)
                 
        n = self.n_successive_samples
        batch_x = np.zeros((current_batch_size * n, self.target_size[0], self.target_size[1], 3), dtype=K.floatx())
        batch_y = np.zeros((current_batch_size * n, 5)) if self.output_counts\
        else np.zeros((current_batch_size * n, self.target_size[0], self.target_size[1], 5), dtype=K.floatx())
        
        # For each index, we load the data and sample randomly n_successive_samples patches
        for i, j in enumerate(index_array):
            index = j // self.n_blocks_per_batch
            image_id = self.image_ids[index]
            block_id = self.block_ids[index]
            print("Imid, bid: ", image_id, block_id)
            uid = "{iid}_{bid}".format(iid=image_id, bid=block_id)
            img = cv2.imread(os.path.join(self.root_dir, "TrainBlock", uid + ".jpg"))
            dmap = np.load(os.path.join(self.root_dir, "TrainDensity", uid + ".npz"))['dmap']
            smap = np.load(os.path.join(self.root_dir, "TrainSampling", uid + ".npz"))['smap']
            
            xs, ys = self.__sample__(img, dmap, smap, n)
            batch_x[i * n: (i + 1) * n,:,:,:] = xs
            if not self.output_counts: 
                batch_y[i * n: (i + 1) * n,:,:,:] = ys
            else:
                batch_y[i * n: (i + 1) * n,:] = np.sum(np.sum(ys, axis=1), axis=1)          

        return self.__normalize_input__(batch_x), self.__normalize_output__(batch_y)

In [126]:
mean_bgr = np.load("../data/sealion/mean_bgr.npy")
std_bgr = np.load("../data/sealion/std_bgr.npy")
patchesGenerator = PatchIterator("/home/lowik/sealion/data/sealion/", [0, 1], mean_bgr, std_bgr, 1)

In [127]:
def show_density(im, density):
    max_density = np.max(density)
    if max_density > 0:
        normalized_density = density/max_density
    else:
        normalized_density = density
    im_fp32 = im.astype(np.float32)
    im_masked = im_fp32.copy()
    im_masked[:,:,0] = (im_fp32[:,:,0] * normalized_density)
    im_masked[:,:,1] = (im_fp32[:,:,1] * normalized_density)
    im_masked[:,:,2] = (im_fp32[:,:,2] * normalized_density)
    return im_masked.astype(np.uint8), (normalized_density * 255).astype(np.uint8)

In [130]:
for batch_x, batch_y in patchesGenerator:
    for i, (x, y) in enumerate(zip(batch_x, batch_y)):
        im, mask = show_density(x, y[:,:,2])
        cv2.imwrite("../data/dmap" + str(i) + ".png", mask)
    break

Imid, bid:  0 4
x, y:  1432 964
x, y:  935 212
x, y:  1332 847
x, y:  1098 457
Imid, bid:  1 11
x, y:  1483 936
x, y:  1000 192
x, y:  166 1112
x, y:  682 960


In [ ]:
a = np.load("../data/sealion/TrainSampling/3.npz")

In [ ]:
a['arr_0']